# Polarizability using automatic differentiation

Simple example for computing properties using (forward-mode)
automatic differentation.
For a more classical approach and more details about computing polarizabilities,
see Polarizability by linear response.

In [1]:
using DFTK
using LinearAlgebra
using ForwardDiff

# Construct PlaneWaveBasis given a particular electric field strength
# Again we take the example of a Helium atom.
function make_basis(ε::T; a=10., Ecut=30) where T
    lattice=T(a) * I(3)  # lattice is a cube of ``a`` Bohrs
    # Helium at the center of the box
    atoms     = [ElementPsp(:He, psp=load_psp("hgh/lda/He-q2"))]
    positions = [[1/2, 1/2, 1/2]]

    model = model_DFT(lattice, atoms, positions, [:lda_x, :lda_c_vwn];
                      extra_terms=[ExternalFromReal(r -> -ε * (r[1] - a/2))],
                      symmetries=false)
    PlaneWaveBasis(model; Ecut, kgrid=[1, 1, 1])  # No k-point sampling on isolated system
end

# dipole moment of a given density (assuming the current geometry)
function dipole(basis, ρ)
    @assert isdiag(basis.model.lattice)
    a  = basis.model.lattice[1, 1]
    rr = [a * (r[1] - 1/2) for r in r_vectors(basis)]
    sum(rr .* ρ) * basis.dvol
end

# Function to compute the dipole for a given field strength
function compute_dipole(ε; tol=1e-8, kwargs...)
    scfres = self_consistent_field(make_basis(ε; kwargs...), tol=tol)
    dipole(scfres.basis, scfres.ρ)
end;

With this in place we can compute the polarizability from finite differences
(just like in the previous example):

In [2]:
polarizability_fd = let
    ε = 0.01
    (compute_dipole(ε) - compute_dipole(0.0)) / ε
end

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -2.770913397474                   -0.52    9.0
  2   -2.772149612126       -2.91       -1.32    1.0
  3   -2.772169910421       -4.69       -2.37    2.0
  4   -2.772170722591       -6.09       -4.00    2.0
  5   -2.772170723008       -9.38       -5.00    2.0
n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -2.770751990144                   -0.53    8.0
  2   -2.772047381018       -2.89       -1.31    1.0
  3   -2.772082596381       -4.45       -2.61    1.0
  4   -2.772083416150       -6.09       -4.09    2.0
  5   -2.772083417754       -8.79       -4.64    2.0


1.7732661269077885

We do the same thing using automatic differentiation. Under the hood this uses
custom rules to implicitly differentiate through the self-consistent
field fixed-point problem.

In [3]:
polarizability = ForwardDiff.derivative(compute_dipole, 0.0)
println()
println("Polarizability via ForwardDiff:       $polarizability")
println("Polarizability via finite difference: $polarizability_fd")

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -2.770735900316                   -0.53    8.0
  2   -2.772052363233       -2.88       -1.31    1.0
  3   -2.772082720727       -4.52       -2.56    1.0
  4   -2.772083402296       -6.17       -3.51    2.0
  5   -2.772083417177       -7.83       -4.08    2.0
  6   -2.772083417807       -9.20       -4.97    2.0

Polarizability via ForwardDiff:       1.7725254937958959
Polarizability via finite difference: 1.7732661269077885
